# Import needed modules and useful functions

In [ ]:
#-- useful python script for training the DeepTICA cvs --#
from utils import *

np.random.seed(13)

#-- to not visualize warnings --#
import warnings
warnings.filterwarnings('ignore')

# Simulation parameters 

In [ ]:
kb=0.008314
#-- SIMULATION PARAMETERS --#
sim_parameters = {
    'temp':300, 
    'beta': 1./(300*kb),
    'kbt': None,
    #-- parameters to compute the fes --#
    'blocks':2,
    'bandwidth': 0.02,
    'plot_max_fes' :70,
}
#--------------------------------------#

# Prepare input files for plumed

In [ ]:
folder = "angles/"
Path(folder).mkdir(parents=True, exist_ok=True)

with open(folder+"plumed.dat","w") as file:
    print("""
# vim:ft=plumed

MOLINFO STRUCTURE=input.ala2.pdb
phi: TORSION ATOMS=@phi-2
psi: TORSION ATOMS=@psi-2
ene: ENERGY

INCLUDE FILE=plumed_descriptors.data

opes: OPES_METAD ARG=phi,psi TEMP=300 PACE=500 FILE=KERNELS RESTART=NO BARRIER=40 STATE_WFILE=RestartKernels STATE_WSTRIDE=500*10

PRINT FMT=%g STRIDE=500 FILE=COLVAR ARG=*

ENDPLUMED
""",file=file)

#-- run gromacs --#
#execute("cp script/input.* script/plumed_descriptors.data "+folder,folder=".")
#execute("./run_gromacs.sh",folder=folder)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
data = load_dataframe(folder+"COLVAR")
load_dataframe(folder+"COLVAR").plot.scatter(y="psi",x="phi",c="opes.bias",cmap="fessa",ax=ax)
ax.set_xlabel(r"$\phi$")
ax.set_ylabel(r"$\psi$")
fig.savefig(folder+"images/"+"traj.png",transparent=False,facecolor="white",dpi=300)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
data.hist(column="opes.bias",ax=ax)
ax.set_yscale("log")
fig.savefig(folder+"images/"+"bias_hist.png",transparent=False,facecolor="white",dpi=300)

fig,ax = plt.subplots(1,1,figsize=(8,6))
data.hist(column="opes.bias",ax=ax,cumulative=True)
ax.set_yscale("log")

fig,ax = plt.subplots(1,1,figsize=(8,6))
data["opes.bias"].plot.kde(ax=ax)
#ax.set_yscale("log")

fig,ax = plt.subplots(1,1,figsize=(8,6))
data = load_dataframe(folder+"COLVAR")
print(33.28968972894314 - max(data["opes.bias"]))
delta = 33.28968972894314 - max(data["opes.bias"])
#data["opes.bias"] -= (data["opes.bias"].min())
data["opes.bias"] += delta 
#data["opes.bias"] *= -1
data.plot.scatter(x="phi",y="opes.bias",ax=ax)

In [ ]:
# ref from long unbias 1555270 ps

# I want to estimate the escape time from basin A to basin B and viceversa
# the idea is the following: take the time series and divide the time series in this way 
# interval for phi : [-pi,0] and [0,pi]
data = load_dataframe(folder+"COLVAR")
data["opes.bias"] += delta#(40-max(data["opes.bias"]))
from_time = 0#30000
data = data[from_time:]

print(len(data))
data = data[:]
print(len(data))

indeces_basinA = ( (data["phi"] < 0) ) #| (data["phi"] > 3) ) #& ( (data["psi"] > -1) | (data["psi"] < -2.5) ) )
data[indeces_basinA].plot.scatter(x="phi",y="psi")
data[indeces_basinA].hist(column="opes.bias")
#print(data[indeces_basinA].index) 
#indeces_basinB = ( (data["phi"] > 0) & (data["phi"] < np.pi) )

time = 0
times = []
prev_index = -1 
dt = 1#data["time"][1]-data["time"][0]

for index in (data[indeces_basinA].index-from_time):
    #print(index)
    if index == prev_index+1:
        time += dt * np.exp( sim_parameters["beta"] * data["opes.bias"].to_numpy()[index] )
    else:
        times.append(time)
        time = 0
    prev_index = index
#print(times)
times = pd.DataFrame(times,columns=["times"])
times = times[times["times"]>1000000] # problema nel definire la regione in cui calcolare questo elapsed time. E allo stesso tempo
                                      # su come riscalare il potenziale
print(times[1:].max())
print(times[1:].min())
print(times[1:].head())

print( times[1:].std() , times[1:].mean())

ref = 1555270 
print(times.mean()-ref)

In [ ]:
'''
data = data[data["opes.bias"]>-5]
fig,ax = plt.subplots(1,1,figsize=(8,6))
data.plot.scatter(y="psi",x="phi",c="opes.bias",cmap="fessa",ax=ax)
ax.set_xlabel(r"$\phi$")
ax.set_ylabel(r"$\psi$")
fig.savefig(folder+"images/"+"traj.png",transparent=False,facecolor="white",dpi=300)
'''

In [ ]:
#-- estimation of Free Energy Surface --#
s = load_dataframe(folder+"COLVAR").filter(regex="^p").to_numpy()
data = load_dataframe(folder+"COLVAR")
logweight=( data["opes.bias"].to_numpy()-max(data["opes.bias"].to_numpy()) )*sim_parameters["beta"]

''' questo riscalamento che uso non dà una stima vera delle free energy. Infatti è come se cambiassi la temperatura
    e ottengo qualcosa molto simile a non usare i pesi, cioè stime di una simulazione unbias come se non l'avessi nemmeno biasiata
logweight=( data["opes.bias"].to_numpy()-max(data["opes.bias"].to_numpy()) )
logweight /= np.abs(np.min(logweight))
logweight *= sim_parameters["beta"]
'''

fig, ax = plt.subplots(figsize=(8,8))
for i in range(2):
    fes,grid,bounds,error = compute_fes(s[:,i], weights=np.exp(logweight),
                                        temp=sim_parameters["temp"],
                                        kbt=sim_parameters["kbt"],
                                        blocks=sim_parameters["blocks"],
                                        bandwidth=sim_parameters["bandwidth"],scale_by='range',
                                        plot=True, plot_max_fes=sim_parameters["plot_max_fes"], ax = ax)
    condition = (grid < 0.04) & (grid > 0.03)
    print(np.where(condition))
    print(fes[50])

data = load_dataframe(folder+"COLVAR")
delta = 33.28968972894314 - max(data["opes.bias"])
data["opes.bias"] += delta 
data.plot.scatter(x="phi",y="opes.bias",ax=ax,c="yellow",zorder=-1)

ax.legend([r"$F(\phi)$ estimate",r"deposited bias",r"$F(\psi)$ estimate"])   
ax.grid()
ax.set_xlabel(r"$(\phi,\psi)$")
ax.set_ylabel("FES [Kj/mol]")
plt.tight_layout()
fig.savefig(folder+"images/"+"FreeEnergy_notW.png",transparent=False,facecolor="white",dpi=300)
plt.show()

In [ ]:
#data["opes.bias"] -= np.max(data["opes.bias"])
data[:].plot.scatter(x="time",y="opes.bias")

In [ ]:
#data = load_dataframe(folder+"COLVAR")
#data = data[data["time"]>20000]
#data = data[data["opes.bias"]>-5]

In [ ]:
#''' compare different set up for rescaled time
fig1,axs1 = plt.subplots(1,3,figsize=(25,4))
fig2,axs2 = plt.subplots(1,3,figsize=(25,4))
fig3,axs3 = plt.subplots(1,3,figsize=(25,4))
fig4,axs4 = plt.subplots(1,3,figsize=(25,4))
fig5,axs5 = plt.subplots(1,3,figsize=(25,4))
fig6,axs6 = plt.subplots(1,3,figsize=(25,4))
fig7,axs7 = plt.subplots(1,3,figsize=(25,6))
fig8,axs8 = plt.subplots(1,3,figsize=(25,4))
    
X = data.filter(regex='^d[^a-z]').values
# rescaled as must be
logweight = data["opes.bias"].to_numpy()*sim_parameters["beta"]
t = data['time'].values
dt = t[1]-t[0]
tprime = dt * np.cumsum(np.exp(logweight))
axs1[0].plot(t,np.exp(logweight))
axs1[0].set_title("(w,t) w")
axs2[0].plot(t,tprime)
axs2[0].set_title("(t',t) w")
axs3[0].plot(t[1:],tprime[1:]/t[1:])
axs3[0].set_title("(t'/t, t) w")
axs5[0].scatter(tprime[:1000],data["phi"].to_numpy()[:1000])
axs5[0].set_title("(t',phi)")
deltatprime = [tprime[i+1]-tprime[i] for i in range(len(tprime)-1)]
axs6[0].scatter(t[1:],deltatprime)
axs6[0].set_title("(t,deltat')")
bins = np.logspace(-8,1)
hist,edges,patches = axs7[0].hist(deltatprime,bins)
print(edges[0],edges[-1])
print(np.max(hist))
axs7[0].set_title(r"Histogram of $\Delta t$' (1)")
axs7[0].set_xscale("log")
axs7[0].set_yscale("log")
axs7[0].set_xlabel("Deltatprime")
axs7[0].set_ylabel("Counts")
deltaprime = deltatprime.insert(0,0)
data["deltatprime"] = deltatprime
data.plot.scatter(x="phi",y="psi",c="deltatprime",s=1,cmap="fessa",ax=axs8[0])

# rescaled as Michele wants
logweight -= max(logweight)
tprime = dt * np.cumsum(np.exp(logweight))
axs1[1].plot(t,np.exp(logweight))
axs1[1].set_title("(w,t) w-max()")
axs2[1].plot(t,tprime)
axs2[1].set_title("(t',t) w-max()")
axs3[1].plot(t[1:],tprime[1:]/t[1:])
axs3[1].set_title("(t'/t,t) w-max()")
axs5[1].scatter(tprime[:1000],data["phi"].to_numpy()[:1000])
axs5[1].set_title("(t',phi) w-max()")
deltatprime = [tprime[i+1]-tprime[i] for i in range(len(tprime)-1)]
axs6[1].scatter(t[1:],deltatprime)
axs6[1].set_title("(t,deltat') w-max()")
bins = np.logspace(-9,1.5)
hist,edges,patches = axs7[1].hist(deltatprime,bins)
print(np.max(hist))
axs7[1].set_title(r"Histogram of $\Delta t$' (1)")
axs7[1].set_xscale("log")
axs7[1].set_yscale("log")
axs7[1].set_xlabel("Deltatprime")
axs7[1].set_ylabel("Counts")
deltaprime = deltatprime.insert(0,0)
data["deltatprime"] = deltatprime
data.plot.scatter(x="phi",y="psi",c="deltatprime",s=1,cmap="fessa",ax=axs8[1])

# rescaled as I want
logweight = data["opes.bias"].to_numpy()-max(data["opes.bias"].to_numpy())
#logweight /= np.abs(np.min(logweight))
logweight *= sim_parameters["beta"]
logweight /= np.abs(np.min(logweight))
tprime = dt * np.cumsum(np.exp(logweight))
axs1[2].plot(t,np.exp(logweight))
axs1[2].set_title("(w,t) (w-max())/np.abs(min)")
axs2[2].plot(t,tprime)
axs2[2].set_title("(t',t) (w-max())/np.abs(min)")
axs3[2].plot(t[1:],tprime[1:]/t[1:])
axs3[2].set_title("(t'/t,t) (w-max())/np.abs(min)")
axs5[2].scatter(tprime[:1000],data["phi"].to_numpy()[:1000])
axs5[2].set_title("(t',phi) (w-max())/np.abs(min)")
deltatprime = [tprime[i+1]-tprime[i] for i in range(len(tprime)-1)]
axs6[2].scatter(t[1:],deltatprime)
axs6[2].set_title("(t,deltat') (w-max())/np.abs(min)")
bins = np.logspace(-5,0.1)
hist,edges,patches = axs7[2].hist(deltatprime,bins)
print(np.max(hist))
axs7[2].set_title(r"Histogram of $\Delta t$' (1)")
axs7[2].set_xscale("log")
axs7[2].set_yscale("log")
axs7[2].set_xlabel("Deltatprime")
axs7[2].set_ylabel("Counts")
deltaprime = deltatprime.insert(0,0)
data["deltatprime"] = deltatprime
data.plot.scatter(x="phi",y="psi",c="deltatprime",s=1,cmap="fessa",ax=axs8[2])

#-- trajectories in phi / psi --#

fes = np.loadtxt("angles/fes.txt",delimiter=" ")
grid0 = np.loadtxt("angles/grid0.txt",delimiter=" ")
grid1 = np.loadtxt("angles/grid1.txt",delimiter=" ")
bounds = np.arange(0, 60, 5.)
for i in range(3):
    data.plot.scatter(y="psi",x="phi",ax=axs4[i])
    c = axs4[i].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
        norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
    )
    c.clabel()
    axs4[i].grid()
    axs4[i].set_xlabel(r"$\phi$")
    axs4[i].set_ylabel(r"$\psi$")
plt.tight_layout()

fig1.savefig(folder+"images/"+"t_vs_w.png",transparent=False,facecolor="white",dpi=300)
fig2.savefig(folder+"images/"+"t_vs_tprime.png",transparent=False,facecolor="white",dpi=300)
fig3.savefig(folder+"images/"+"tprime:t_vs_t.png",transparent=False,facecolor="white",dpi=300)
fig6.savefig(folder+"images/"+"t_vs_deltatprime.png",transparent=False,facecolor="white",dpi=300)
fig7.savefig(folder+"images/"+"histdeltatprime.png",transparent=False,facecolor="white",dpi=300)
fig8.savefig(folder+"images/"+"Scatterdeltatprime.png",transparent=False,facecolor="white",dpi=300)

In [ ]:
'''
fig,axs1 = plt.subplots(1,3,figsize=(20,4))
data = load_dataframe(folder+"COLVAR")
t = data['time'].values
X = data.filter(regex='^d[^a-z]').values
logweight = data["opes.bias"].to_numpy()*sim_parameters["beta"]
dt = t[1]-t[0]
tprime = dt * np.cumsum(np.exp(logweight))
x = data["phi"].values
#-- without minus max --#
autocorr = np.empty(0)
N = 100
lags = np.linspace(0,10,N)
for lag in lags:
    res = my_autocorrelation_python(x,lag=lag,weight=np.exp(logweight),time=t, tprime=tprime)
    autocorr = np.append(autocorr,res)
axs1[0].plot(lags,autocorr)
axs1[0].set_title("(t,w)")
#-- with minus max --#
autocorr = np.empty(0)
logweight -= max(logweight)
tprime = dt * np.cumsum(np.exp(logweight))
for lag in lags:
    res = my_autocorrelation_python(x,lag=lag,weight=np.exp(logweight),time=t, tprime=tprime)
    autocorr = np.append(autocorr,res)
axs1[1].plot(lags,autocorr)
axs1[1].set_title("(t,w) w-max() ")
#-- with minus max and / np.abs(min)--#
logweight = data["opes.bias"].to_numpy()-max(data["opes.bias"].to_numpy())
logweight /= np.abs(np.min(logweight))
autocorr = np.empty(0)
logweight *= sim_parameters["beta"]
tprime = dt * np.cumsum(np.exp(logweight))
for lag in lags:
    res = my_autocorrelation_python(x,lag=lag,weight=np.exp(logweight),time=t, tprime=tprime)
    autocorr = np.append(autocorr,res)
axs1[2].plot(lags,autocorr)
axs1[2].set_title("(t,w) (w-max())/np.abs(min)")
'''

In [ ]:
descriptors_names = data.filter(regex='^p').columns.values

fig,axs = plt.subplots(1,2,figsize=(8,4),sharey=True)

for ax,desc in zip(axs.flatten(),descriptors_names):
    data[desc].plot.hist(bins=50,alpha=1,ax=ax,legend=False,grid=True,histtype='step',linewidth=2,density=True)
    data[desc].plot.hist(bins=50,alpha=0.5,ax=ax,legend=False,grid=True,color="grey",density=True)
    ax.set_title(desc)

fig,axs = plt.subplots(1,2,figsize=(12,6))

for ax,desc in zip(axs.flatten(),descriptors_names):
    data[::1].plot.scatter(x="time",y=desc,alpha=1,ax=ax,legend=False,grid=True,linewidth=2,marker="^")
    #data[::100].plot.line(x="time",y=desc,alpha=1,ax=ax,legend=False,grid=True,color="grey")
    ax.set_title(desc)

fig.savefig(folder+"images/"+"phi_psi.png",transparent=False,facecolor="white",dpi=300)

plt.tight_layout()

## Deep TICA Analysis  
Qui analizzo con la Deeptica la simulazione biased.  
Lo farò utilizzando 4 diversi tprime per il tempo riscalato: 
1. senza usare il tempo riscalato 
2. usando il vero e proprio tempo riscalato con tprime = dt * exp(beta*logweights)
3. come farebbe michele con tprime = dt * exp(beta*logweights-max(beta*logweights))
4. come farei io con logweights -= max(logweights); logweights /= np.abs(np.min(logweights)); logweights \*= beta, tprime = dt * exp(logweights)

Results: 
Per tempi molti grandi nessuno degli algoritmi lavora bene. Per tempi piccoli, sulla scala del tempo di stride, lavora bene *1* e *4*, su tempi molto piccoli nessuno lavora bene.  
Facendo a batches tra 1, 5 con 5 valori:   
# 1

In [ ]:
torch.manual_seed(21)
data = load_dataframe(folder+"COLVAR")[:]
size = len(data)
min_lag,max_lag = 1,2 
n = 1 # how many lag times between min and max lag
lags = np.linspace(min_lag,max_lag,n) #-- how many batches for the train and valid set of a single simulation
print(lags)
shuffle = False # if shuffle the data between batches
#-- train_datasets and valid_datasets list, it will be filled with new data every iteration
train_datasets = []
valid_datasets = []
# torch seed 
torch.manual_seed(21)

descriptors_names = data.filter(regex='^d[^a-z]').columns.values

#-- TRAINING PARAMETERS --#
n_output = 2 # 2 non linear combination of the descriptors  
n_input = len(descriptors_names) # can change..
train_parameters = {
              'descriptors': '^d[^a-z]', # can change during simulation
              'nodes':[n_input,30,30,n_output],
              'activ_type': 'tanh',#'relu','selu','tanh'
              'lag_time':10, 
              'loss_type': 'sum', 
              'n_eig': n_output,
              'trainsize':0.7, 
              'lrate':1e-3,
              'l2_reg':0.,
              'num_epochs':1000,
              'batchsize': -1, #---> è da fare sul train loder and valid loader
              'es_patience':1000,
              'es_consecutive':True,
              'standardize_outputs':True,
              'standardize_inputs': True,
              'log_every':50,
              }

# how many data in single batch, batchsize
n_train = int( size*train_parameters["trainsize"] )
n_valid = int( size*(1-train_parameters["trainsize"])-int(10*max_lag) )
print("training samples: ",n_train, "\t validation samples", n_valid)

# DEVICE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

t = data['time'].values
X = data[descriptors_names].values
train_parameters['num_epochs'] = 400

In [ ]:
#print(torch.get_num_threads())
#torch.set_num_threads(16)
#print(torch.get_num_interop_threads())
#torch.set_num_interop_threads(16)

In [ ]:
# create time lagged dataset with different lag times
for lag in lags:
    #random split
    # TensorDataset (x_t,x_lag,w_t,w_lag)
    dataset = create_time_lagged_dataset(X,t=t,lag_time=lag,interval=[0,n_train+n_valid])
    print(len(dataset))
    # save correlated couples and weights
    x_t,x_lag,w_t,w_lag = np.array(dataset[:][0]),np.array(dataset[:][1]),np.array(dataset[:][2]),np.array(dataset[:][3])
    train_data, valid_data = random_split(dataset,[n_train,n_valid])
    train_datasets.append(train_data)
    valid_datasets.append(valid_data)

train_loader = FastTensorDataLoader(train_datasets, batch_size=n_train,shuffle=shuffle)
valid_loader = FastTensorDataLoader(valid_datasets, batch_size=n_valid,shuffle=shuffle)

#-- TRAIN --#
# MODEL
model = DeepTICA_CV(train_parameters['nodes'],activation=train_parameters['activ_type'],gaussian_random_initialization=True)
model.to(device)
# OPTIMIZER (Adam)
opt = torch.optim.Adam(model.parameters(), lr=train_parameters['lrate'], weight_decay=train_parameters['l2_reg'])
# lrscheduler
#model.set_LRScheduler(opt,min_lr=5e-5)
model.set_optimizer(opt)
if valid_loader is not None:
    # EarlyStopping
    model.set_earlystopping(patience=train_parameters['es_patience'],
                            min_delta=0.005,consecutive=train_parameters['es_consecutive'], save_best_model=True, log=False) 
# TRAIN
model.fit(train_loader=train_loader,valid_loader=valid_loader,
    standardize_inputs=train_parameters['standardize_inputs'],
    standardize_outputs=train_parameters['standardize_outputs'],
    loss_type=train_parameters['loss_type'],
    n_eig=train_parameters['n_eig'],
    nepochs=train_parameters['num_epochs'],
    info=False, log_every=train_parameters['log_every'])
#-- move the model back to cpu for convenience --#
model.to('cpu')

In [ ]:
dataset = create_time_lagged_dataset(X,t=t,lag_time=1000)
print(len(dataset))
# save correlated couples and weights
x_t,x_lag,w_t,w_lag = np.array(dataset[:][0]),np.array(dataset[:][1]),np.array(dataset[:][2]),np.array(dataset[:][3])
print(len(x_t))
indeces_t_1 = [ data.index[ np.abs(data[descriptors_names].to_numpy().T[0]-el) < 1e-6 ][0] for el in x_t.T[0] ]
indeces_lag_1 = [ data.index[ np.abs(data[descriptors_names].to_numpy().T[0]-el) < 1e-6 ][0] for el in x_lag.T[0] ]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_1].plot.scatter(x="phi",y="psi",ax=ax,c="red")
data.iloc[indeces_lag_1].plot.scatter(x="phi",y="psi",ax=ax,c="blue",alpha=0.5)
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_1]["phi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_1]["phi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$\phi$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_1]["psi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_1]["psi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$\psi$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_1]["ene"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_1]["ene"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$Energy$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_1][descriptors_names].plot.hist(ax=ax,color="red",density=True,histtype='step',linewidth=2,legend=False)
data.iloc[indeces_lag_1][descriptors_names].plot.hist(ax=ax,histtype='step',density=True,color="blue",linewidth=2,legend=False)
ax.set_yscale("log")

#time series
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_1].plot.scatter(x="time",y="phi",ax=ax,color="red")
data.iloc[indeces_lag_1].plot.scatter(x="time",y="phi",ax=ax,color="blue",marker="^")
ax.set_ylabel(r"$\phi$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_1].plot.scatter(x="time",y="psi",ax=ax,color="red")
data.iloc[indeces_lag_1].plot.scatter(x="time",y="psi",ax=ax,color="blue",marker="^")
ax.set_ylabel(r"$\psi$")

In [ ]:
plot_model_lossfunction(model)

In [ ]:
# plot cvs #
fig,ax = plt.subplots(1,2,figsize=(12,6),sharey=True)

data["cv1"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[0]
data["cv2"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[1]   

data.plot.hexbin(y="psi",x="phi",C="cv1",cmap="fessa",ax=ax[0])
data.plot.hexbin(y="psi",x="phi",C="cv2",cmap="fessa",ax=ax[1])

fes = np.loadtxt("angles/fes.txt",delimiter=" ")
grid0 = np.loadtxt("angles/grid0.txt",delimiter=" ")
grid1 = np.loadtxt("angles/grid1.txt",delimiter=" ")
bounds = np.arange(0, 60, 5.)
c = ax[0].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c = ax[1].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c.clabel()
ax[0].grid()
ax[0].set_xlabel(r"$\phi$")
ax[0].set_ylabel(r"$\psi$")
ax[0].set_title("Deep TICA 1")
ax[1].grid()
ax[1].set_xlabel(r"$\phi$")
ax[1].set_ylabel(r"$\psi$")
ax[1].set_title("Deep TICA 2")

fig.savefig(folder+"images/"+"cv_t.png",transparent=False,facecolor="white",dpi=300)

# 2

In [ ]:
torch.manual_seed(21)
data = load_dataframe(folder+"COLVAR")
train_parameters = {
              'descriptors': '^d[^a-z]', # can change during simulation
              'nodes':[n_input,30,30,n_output],
              'activ_type': 'tanh',#'relu','selu','tanh'
              'lag_time':10, 
              'loss_type': 'sum', 
              'n_eig': n_output,
              'trainsize':0.7, 
              'lrate':5e-4,
              'l2_reg':0.,
              'num_epochs':500,
              'batchsize': -1, #---> è da fare sul train loder and valid loader
              'es_patience':100,
              'es_consecutive':True,
              'standardize_outputs':True,
              'standardize_inputs': True,
              'log_every':50,
              }

# evaluate tprime
logweight = data["opes.bias"].to_numpy()*sim_parameters["beta"]
dt = t[1]-t[0]
tprime = dt * np.cumsum(np.exp(logweight))
X = data[descriptors_names].to_numpy()
#lags = [6]
min_lag,max_lag = 1,1#0.01,25
n = 1#50 # how many lag times between min and max lag
lags = np.linspace(min_lag,max_lag,n) #-- how many batches for the train and valid set of a single simulation
print(lags)

In [ ]:
from mlcvs.utils.data import create_time_lagged_dataset_new

train_datasets = []
valid_datasets = []
# create time lagged dataset with different lag times
for lag in lags:
    #random split
    # TensorDataset (x_t,x_lag,w_t,w_lag)
    dataset = create_time_lagged_dataset_new(X,t=t,lag_time=lag,tprime=tprime,logweights=logweight,interval=[0,n_train+n_valid])
    print(len(dataset))
    x_t,x_lag,w_t,w_lag = np.array(dataset[:][0]),np.array(dataset[:][1]),np.array(dataset[:][2]),np.array(dataset[:][3])
    train_data, valid_data = random_split(dataset,[n_train,n_valid])
    #train_data = Subset(dataset,np.arange(0,n_valid) )
    #valid_data = Subset(dataset,np.arange(n_valid,n_train+n_valid) )
    train_datasets.append(train_data)
    valid_datasets.append(valid_data)

train_loader = FastTensorDataLoader(train_datasets, batch_size=n_train,shuffle=shuffle)
valid_loader = FastTensorDataLoader(valid_datasets, batch_size=n_valid,shuffle=shuffle)

#-- TRAIN --#
# MODEL
model = DeepTICA_CV(train_parameters['nodes'],activation=train_parameters['activ_type'],gaussian_random_initialization=True)
model.to(device)
# OPTIMIZER (Adam)
opt = torch.optim.Adam(model.parameters(), lr=train_parameters['lrate'], weight_decay=train_parameters['l2_reg'])
# lrscheduler
#model.set_LRScheduler(opt,min_lr=5e-5)
model.set_optimizer(opt)
if valid_loader is not None:
    # EarlyStopping
    model.set_earlystopping(patience=train_parameters['es_patience'],
                            min_delta=0.005,consecutive=train_parameters['es_consecutive'], save_best_model=True, log=False) 
# TRAIN
model.fit(train_loader=train_loader,valid_loader=valid_loader,
    standardize_inputs=train_parameters['standardize_inputs'],
    standardize_outputs=train_parameters['standardize_outputs'],
    loss_type=train_parameters['loss_type'],
    n_eig=train_parameters['n_eig'],
    nepochs=train_parameters['num_epochs'],
    info=False, log_every=train_parameters['log_every'])
#-- move the model back to cpu for convenience --#
model.to('cpu')

In [ ]:
print(len(x_t))
indeces_t_2 = [ data.index[ np.abs(data[descriptors_names].to_numpy().T[0]-el) < 1e-6 ][0] for el in x_t.T[0] ]
indeces_lag_2 = [ data.index[ np.abs(data[descriptors_names].to_numpy().T[0]-el) < 1e-6 ][0] for el in x_lag.T[0] ]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_2].plot.scatter(x="phi",y="psi",ax=ax,c="red")
data.iloc[indeces_lag_2].plot.scatter(x="phi",y="psi",ax=ax,c="blue",alpha=0.5)
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_2]["phi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_2]["phi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$\phi$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_2]["psi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_2]["psi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$\psi$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_2]["ene"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_2]["ene"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$Energy$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_2][descriptors_names].plot.hist(ax=ax,color="red",density=True,histtype='step',linewidth=2,legend=False)
data.iloc[indeces_lag_2][descriptors_names].plot.hist(ax=ax,histtype='step',density=True,color="blue",linewidth=2,legend=False)
ax.set_yscale("log")

#time series
fig,ax = plt.subplots(1,1,figsize=(16,8))
data.iloc[indeces_t_2].plot.scatter(x="time",y="phi",ax=ax,color="red")
data.iloc[indeces_lag_2].plot.scatter(x="time",y="phi",ax=ax,color="blue",marker="^")
ax.set_ylabel(r"$\phi$")
fig,ax = plt.subplots(1,1,figsize=(16,8))
data.iloc[indeces_t_2].plot.scatter(x="time",y="psi",ax=ax,color="red")
data.iloc[indeces_lag_2].plot.scatter(x="time",y="psi",ax=ax,color="blue",marker="^")
ax.set_ylabel(r"$\psi$")

In [ ]:
plot_model_lossfunction(model)

In [ ]:
X = data[descriptors_names].to_numpy()

# plot cvs #
data["cv1"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[0]
data["cv2"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[1]   

fig,ax = plt.subplots(1,2,figsize=(12,6))
data.plot.hexbin(y="psi",x="phi",C="cv1",cmap="viridis",ax=ax[0])
data.plot.hexbin(y="psi",x="phi",C="cv2",cmap="viridis",ax=ax[1])

fes = np.loadtxt("angles/fes.txt",delimiter=" ")
grid0 = np.loadtxt("angles/grid0.txt",delimiter=" ")
grid1 = np.loadtxt("angles/grid1.txt",delimiter=" ")
bounds = np.arange(0, 60, 5.)
c = ax[0].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c = ax[1].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c.clabel()
ax[0].grid()
ax[0].set_xlabel(r"$\phi$")
ax[0].set_ylabel(r"$\psi$")
ax[0].set_title("Deep TICA 1")
ax[1].grid()
ax[1].set_xlabel(r"$\phi$")
ax[1].set_ylabel(r"$\psi$")
ax[1].set_title("Deep TICA 2")

fig.savefig(folder+"images/"+"cv_tprime_w.png",transparent=False,facecolor="white",dpi=300)

# 3 

In [ ]:
data = load_dataframe(folder+"COLVAR")

torch.manual_seed(21)
train_parameters = {
              'descriptors': '^d[^a-z]', # can change during simulation
              'nodes':[n_input,30,30,n_output],
              'activ_type': 'tanh',#'relu','selu','tanh'
              'lag_time':10, 
              'loss_type': 'sum', 
              'n_eig': n_output,
              'trainsize':0.7, 
              'lrate':1e-3,
              'l2_reg':0.,
              'num_epochs':400,
              'batchsize': -1, #---> è da fare sul train loder and valid loader
              'es_patience':100,
              'es_consecutive':True,
              'standardize_outputs':True,
              'standardize_inputs': True,
              'log_every':50,
              }

# evaluate tprime
logweight =  data["opes.bias"].to_numpy()*sim_parameters["beta"]-np.max(data["opes.bias"].to_numpy()*sim_parameters["beta"] )
dt = t[1]-t[0]
tprime = dt * np.cumsum(np.exp(logweight))
#lags = [6]
min_lag,max_lag = 1,1#0.01,25
n = 1#50 # how many lag times between min and max lag
lags = np.linspace(min_lag,max_lag,n) #-- how many batches for the train and valid set of a single simulation
print(lags)

min_lag,max_lag = 1,1#0.001,1
n = 1#50 # how many lag times between min and max lag
lags = np.linspace(min_lag,max_lag,n) #-- how many batches for the train and valid set of a single simulation
print(lags)

In [ ]:
train_datasets = []
valid_datasets = []
# create time lagged dataset with different lag times
for lag in lags:
    #random split
    # TensorDataset (x_t,x_lag,w_t,w_lag)
    dataset = create_time_lagged_dataset_new(X,t=t,lag_time=lag,tprime=tprime,logweights=logweight,interval=[0,n_train+n_valid])
    x_t,x_lag,w_t,w_lag = np.array(dataset[:][0]),np.array(dataset[:][1]),np.array(dataset[:][2]),np.array(dataset[:][3])
    train_data, valid_data = random_split(dataset,[n_train,n_valid])
    train_datasets.append(train_data)
    valid_datasets.append(valid_data)

train_loader = FastTensorDataLoader(train_datasets, batch_size=n_train,shuffle=shuffle)
valid_loader = FastTensorDataLoader(valid_datasets, batch_size=n_valid,shuffle=shuffle)

#-- TRAIN --#
# MODEL
model = DeepTICA_CV(train_parameters['nodes'],activation=train_parameters['activ_type'],gaussian_random_initialization=True)
model.to(device)
# OPTIMIZER (Adam)
opt = torch.optim.Adam(model.parameters(), lr=train_parameters['lrate'], weight_decay=train_parameters['l2_reg'])
# lrscheduler
#model.set_LRScheduler(opt,min_lr=5e-5)
model.set_optimizer(opt)
if valid_loader is not None:
    # EarlyStopping
    model.set_earlystopping(patience=train_parameters['es_patience'],
                            min_delta=0.005,consecutive=train_parameters['es_consecutive'], save_best_model=True, log=False) 
# TRAIN
model.fit(train_loader=train_loader,valid_loader=valid_loader,
    standardize_inputs=train_parameters['standardize_inputs'],
    standardize_outputs=train_parameters['standardize_outputs'],
    loss_type=train_parameters['loss_type'],
    n_eig=train_parameters['n_eig'],
    nepochs=train_parameters['num_epochs'],
    info=False, log_every=train_parameters['log_every'])
#-- move the model back to cpu for convenience --#
model.to('cpu')

In [ ]:
print(len(x_t))
indeces_t_3 = [ data.index[ np.abs(data[descriptors_names].to_numpy().T[0]-el) < 1e-6 ][0] for el in x_t.T[0] ]
indeces_lag_3 = [ data.index[ np.abs(data[descriptors_names].to_numpy().T[0]-el) < 1e-6 ][0] for el in x_lag.T[0] ]

In [ ]:
ffig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_3].plot.scatter(x="phi",y="psi",ax=ax,c="red")
data.iloc[indeces_lag_3].plot.scatter(x="phi",y="psi",ax=ax,c="blue",alpha=0.5)
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_3]["phi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_3]["phi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$\phi$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_3]["psi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_3]["psi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$\psi$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_3]["ene"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_3]["ene"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$Energy$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_3][descriptors_names].plot.hist(ax=ax,color="red",density=True,histtype='step',linewidth=2,legend=False)
data.iloc[indeces_lag_3][descriptors_names].plot.hist(ax=ax,histtype='step',density=True,color="blue",linewidth=2,legend=False)
ax.set_yscale("log")

#time series
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_3].plot.scatter(x="time",y="phi",ax=ax,color="red")
data.iloc[indeces_lag_3].plot.scatter(x="time",y="phi",ax=ax,color="blue",marker="^")
ax.set_ylabel(r"$\phi$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_3].plot.scatter(x="time",y="psi",ax=ax,color="red")
data.iloc[indeces_lag_3].plot.scatter(x="time",y="psi",ax=ax,color="blue",marker="^")
ax.set_ylabel(r"$\psi$")

In [ ]:
plot_model_lossfunction(model)

In [ ]:
X = data[descriptors_names].to_numpy()

# plot cvs #
data["cv1"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[0]
data["cv2"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[1]   

fig,ax = plt.subplots(1,2,figsize=(12,6))
data.plot.hexbin(y="psi",x="phi",C="cv1",cmap="viridis",ax=ax[0])
data.plot.hexbin(y="psi",x="phi",C="cv2",cmap="viridis",ax=ax[1])

fes = np.loadtxt("angles/fes.txt",delimiter=" ")
grid0 = np.loadtxt("angles/grid0.txt",delimiter=" ")
grid1 = np.loadtxt("angles/grid1.txt",delimiter=" ")
bounds = np.arange(0, 60, 5.)
c = ax[0].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c = ax[1].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c.clabel()
ax[0].grid()
ax[0].set_xlabel(r"$\phi$")
ax[0].set_ylabel(r"$\psi$")
ax[0].set_title("Deep TICA 1")
ax[1].grid()
ax[1].set_xlabel(r"$\phi$")
ax[1].set_ylabel(r"$\psi$")
ax[1].set_title("Deep TICA 2")

fig.savefig(folder+"images/"+"cv_tprime_w-max.png",transparent=False,facecolor="white",dpi=300)

# 4

In [ ]:
# evaluate tprime
logweight = data["opes.bias"].to_numpy()-max(data["opes.bias"].to_numpy())
#logweight /= np.abs(min(logweight))
logweight *= sim_parameters["beta"]
logweight /= np.abs(min(logweight))
dt = t[1]-t[0]
tprime = dt * np.cumsum(np.exp(logweight))
min_lag,max_lag = 1,1
n = 1 # how many lag times between min and max lag
lags = np.linspace(min_lag,max_lag,n) #-- how many batches for the train and valid set of a single simulation
train_parameters = {
              'descriptors': '^d[^a-z]', # can change during simulation
              'nodes':[n_input,30,30,n_output],
              'activ_type': 'tanh',#'relu','selu','tanh'
              'lag_time':10, 
              'loss_type': 'sum', 
              'n_eig': n_output,
              'trainsize':0.7, 
              'lrate':1e-3,
              'l2_reg':0.,
              'num_epochs':500,
              'batchsize': -1, #---> è da fare sul train loder and valid loader
              'es_patience':10,
              'es_consecutive':True,
              'standardize_outputs':True,
              'standardize_inputs': True,
              'log_every':50,
              }

In [ ]:
train_datasets = []
valid_datasets = []
# create time lagged dataset with different lag times
for lag in lags:
    #random split
    # TensorDataset (x_t,x_lag,w_t,w_lag)
    dataset = create_time_lagged_dataset(X,t=t,lag_time=np.round(lag,3),tprime=tprime,logweights=logweight,interval=[0,n_train+n_valid])
    x_t,x_lag,w_t,w_lag = np.array(dataset[:][0]),np.array(dataset[:][1]),np.array(dataset[:][2]),np.array(dataset[:][3])
    train_data, valid_data = random_split(dataset,[n_train,n_valid])
    train_datasets.append(train_data)
    valid_datasets.append(valid_data)

train_loader = FastTensorDataLoader(train_datasets, batch_size=n_train,shuffle=shuffle)
valid_loader = FastTensorDataLoader(valid_datasets, batch_size=n_valid,shuffle=shuffle)

#-- TRAIN --#
# MODEL
model = DeepTICA_CV(train_parameters['nodes'],activation=train_parameters['activ_type'],gaussian_random_initialization=True)
model.to(device)
# OPTIMIZER (Adam)
opt = torch.optim.Adam(model.parameters(), lr=train_parameters['lrate'], weight_decay=train_parameters['l2_reg'])
# lrscheduler
#model.set_LRScheduler(opt,min_lr=5e-5)
model.set_optimizer(opt)
if valid_loader is not None:
    # EarlyStopping
    model.set_earlystopping(patience=train_parameters['es_patience'],
                            min_delta=0.005,consecutive=train_parameters['es_consecutive'], save_best_model=True, log=False) 
# TRAIN
model.fit(train_loader=train_loader,valid_loader=valid_loader,
    standardize_inputs=train_parameters['standardize_inputs'],
    standardize_outputs=train_parameters['standardize_outputs'],
    loss_type=train_parameters['loss_type'],
    n_eig=train_parameters['n_eig'],
    nepochs=train_parameters['num_epochs'],
    info=False, log_every=train_parameters['log_every'])
#-- move the model back to cpu for convenience --#
model.to('cpu')

In [ ]:
print(len(x_t))
indeces_t_4 = [ data.index[ np.abs(data[descriptors_names].to_numpy().T[0]-el) < 1e-6 ][0] for el in x_t.T[0] ]
indeces_lag_4 = [ data.index[ np.abs(data[descriptors_names].to_numpy().T[0]-el) < 1e-6 ][0] for el in x_lag.T[0] ]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_4].plot.scatter(x="phi",y="psi",ax=ax,c="red")
data.iloc[indeces_lag_4].plot.scatter(x="phi",y="psi",ax=ax,c="blue",alpha=0.5)

#histograms phi,psi,energy
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_4]["phi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_4]["phi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$\phi$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_4]["psi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_4]["psi"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$\psi$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_4]["ene"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="red")
data.iloc[indeces_lag_4]["ene"].plot.hist(density=False,histtype='step',linewidth=2,ax=ax,bins=100,color="blue",alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel(r"$Energy$")

#histograms descriptors
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_4][descriptors_names].plot.hist(ax=ax,color="red",density=True,histtype='step',linewidth=2,legend=False)
data.iloc[indeces_lag_4][descriptors_names].plot.hist(ax=ax,histtype='step',density=True,color="blue",linewidth=2,legend=False)
ax.set_yscale("log")

#time series
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_4].plot.scatter(x="time",y="phi",ax=ax,color="red")
data.iloc[indeces_lag_4].plot.scatter(x="time",y="phi",ax=ax,color="blue",marker="^")
ax.set_ylabel(r"$\phi$")
fig,ax = plt.subplots(1,1,figsize=(8,8))
data.iloc[indeces_t_4].plot.scatter(x="time",y="psi",ax=ax,color="red")
data.iloc[indeces_lag_4].plot.scatter(x="time",y="psi",ax=ax,color="blue",marker="^")
ax.set_ylabel(r"$\psi$")

In [ ]:
plot_model_lossfunction(model)

In [ ]:
# plot cvs #
data["cv1"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[0]
data["cv2"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[1]   

fig,ax = plt.subplots(1,2,figsize=(12,6))
data.plot.hexbin(y="psi",x="phi",C="cv1",cmap="fessa",ax=ax[0])
data.plot.hexbin(y="psi",x="phi",C="cv2",cmap="fessa",ax=ax[1])

fes = np.loadtxt("angles/fes.txt",delimiter=" ")
grid0 = np.loadtxt("angles/grid0.txt",delimiter=" ")
grid1 = np.loadtxt("angles/grid1.txt",delimiter=" ")
bounds = np.arange(0, 60, 5.)
c = ax[0].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c = ax[1].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c.clabel()
ax[0].grid()
ax[0].set_xlabel(r"$\phi$")
ax[0].set_ylabel(r"$\psi$")
ax[0].set_title("Deep TICA 1")
ax[1].grid()
ax[1].set_xlabel(r"$\phi$")
ax[1].set_ylabel(r"$\psi$")
ax[1].set_title("Deep TICA 2")

fig.savefig(folder+"images/"+"cv_tprime_w-max_divmin.png",transparent=False,facecolor="white",dpi=300)

## TICA Analysis  
Ripeto le stesse analisi di cui sopra ma con la TICA, per capire se la rete può portare ad artefatti   
# 1

In [ ]:
data = load_dataframe(folder+"COLVAR")
t = data['time'].values
X = data[descriptors_names].values

min_lag,max_lag = 1,10 
n = 1 # how many lag times between min and max lag
lags = np.linspace(min_lag,max_lag,n) #-- how many batches for the train and valid set of a single simulation
print(lags)
torch.manual_seed(21)

descriptors_names = data.filter(regex='^d[^a-z]').columns.values

# DEVICE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MODEL
model = TICA_CV(n_features=X.shape[1])
model.to(device)

t = data['time'].values
X = data[descriptors_names].values

In [ ]:
# TRAIN
model.fit_new(X, t, lag=1)

#-- move the model back to cpu for convenience --# 
model.to('cpu')

#-- print some useful results --#
#print("timescales: ",model.tica.timescales(train_parameters["lag_time"]).detach().cpu().numpy()) 
print("eigenvalues: ",model.tica.evals_.detach().cpu().numpy())

In [ ]:
# plot cvs #
fig,ax = plt.subplots(1,2,figsize=(12,6),sharey=True)

data["cv1"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[0]
data["cv2"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[1]   

data.plot.hexbin(y="psi",x="phi",C="cv1",cmap="fessa",ax=ax[0])
data.plot.hexbin(y="psi",x="phi",C="cv2",cmap="fessa",ax=ax[1])

fes = np.loadtxt("angles/fes.txt",delimiter=" ")
grid0 = np.loadtxt("angles/grid0.txt",delimiter=" ")
grid1 = np.loadtxt("angles/grid1.txt",delimiter=" ")
bounds = np.arange(0, 60, 5.)
c = ax[0].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c = ax[1].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c.clabel()
ax[0].grid()
ax[0].set_xlabel(r"$\phi$")
ax[0].set_ylabel(r"$\psi$")
ax[0].set_title("Deep TICA 1")
ax[1].grid()
ax[1].set_xlabel(r"$\phi$")
ax[1].set_ylabel(r"$\psi$")
ax[1].set_title("Deep TICA 2")

fig.savefig(folder+"images/"+"TICAcv_t.png",transparent=False,facecolor="white",dpi=300)

# 2

In [ ]:
data = load_dataframe(folder+"COLVAR")[::5]
t = data['time'].values
X = data[descriptors_names].values

# evaluate tprime
logweight = data["opes.bias"].to_numpy()*sim_parameters["beta"]
dt = t[1]-t[0]
tprime = dt * np.cumsum(np.exp(logweight))
#lags = [6]
min_lag,max_lag = 20,1
n = 1
lags = np.linspace(min_lag,max_lag,n) 
print(lags)

# MODEL
model = TICA_CV(n_features=X.shape[1])
model.to(device)

In [ ]:
# TRAIN
model.fit_new(X, t, lag=5, logweights=logweight,tprime=tprime)

#-- move the model back to cpu for convenience --# 
model.to('cpu')

#-- print some useful results --#
#print("timescales: ",model.tica.timescales(train_parameters["lag_time"]).detach().cpu().numpy()) 
print("eigenvalues: ",model.tica.evals_.detach().cpu().numpy())

In [ ]:
# plot cvs #
data["cv1"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[0]
data["cv2"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[1]   

fig,ax = plt.subplots(1,2,figsize=(12,6))
data.plot.hexbin(y="psi",x="phi",C="cv1",cmap="viridis",ax=ax[0])
data.plot.hexbin(y="psi",x="phi",C="cv2",cmap="viridis",ax=ax[1])

fes = np.loadtxt("angles/fes.txt",delimiter=" ")
grid0 = np.loadtxt("angles/grid0.txt",delimiter=" ")
grid1 = np.loadtxt("angles/grid1.txt",delimiter=" ")
bounds = np.arange(0, 60, 5.)
c = ax[0].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c = ax[1].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c.clabel()
ax[0].grid()
ax[0].set_xlabel(r"$\phi$")
ax[0].set_ylabel(r"$\psi$")
ax[0].set_title("Deep TICA 1")
ax[1].grid()
ax[1].set_xlabel(r"$\phi$")
ax[1].set_ylabel(r"$\psi$")
ax[1].set_title("Deep TICA 2")

fig.savefig(folder+"images/"+"TICAcv_tprime_w.png",transparent=False,facecolor="white",dpi=300)

# 3

Numerical issues with TICA and solve the tica problem. Inversion of C(0)

In [ ]:
data = load_dataframe(folder+"COLVAR")[::5]
t = data['time'].values
X = data[descriptors_names].values

# evaluate tprime
logweight =  (data["opes.bias"].to_numpy()-np.max(data["opes.bias"].to_numpy()) ) *sim_parameters["beta"]
dt = t[1]-t[0]
tprime = dt * np.cumsum(np.exp(logweight))

# MODEL
model = TICA_CV(n_features=X.shape[1])
model.to(device)

In [ ]:
# TRAIN
model.fit_new(X, t, lag=100, logweights=logweight,tprime=tprime)

#-- move the model back to cpu for convenience --# 
model.to('cpu')

#-- print some useful results --#
#print("timescales: ",model.tica.timescales(train_parameters["lag_time"]).detach().cpu().numpy()) 
print("eigenvalues: ",model.tica.evals_.detach().cpu().numpy())

In [ ]:
# plot cvs #
data["cv1"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[0]
data["cv2"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[1]   

fig,ax = plt.subplots(1,2,figsize=(12,6))
data.plot.hexbin(y="psi",x="phi",C="cv1",cmap="viridis",ax=ax[0])
data.plot.hexbin(y="psi",x="phi",C="cv2",cmap="viridis",ax=ax[1])

fes = np.loadtxt("angles/fes.txt",delimiter=" ")
grid0 = np.loadtxt("angles/grid0.txt",delimiter=" ")
grid1 = np.loadtxt("angles/grid1.txt",delimiter=" ")
bounds = np.arange(0, 60, 5.)
c = ax[0].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c = ax[1].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c.clabel()
ax[0].grid()
ax[0].set_xlabel(r"$\phi$")
ax[0].set_ylabel(r"$\psi$")
ax[0].set_title("Deep TICA 1")
ax[1].grid()
ax[1].set_xlabel(r"$\phi$")
ax[1].set_ylabel(r"$\psi$")
ax[1].set_title("Deep TICA 2")

fig.savefig(folder+"images/"+"TICAcv_tprime_w-max.png",transparent=False,facecolor="white",dpi=300)

# 4

In [ ]:
data = load_dataframe(folder+"COLVAR")
t = data['time'].values
X = data[descriptors_names].values

# evaluate tprime
logweight = data["opes.bias"].to_numpy()-max(data["opes.bias"].to_numpy())
#logweight /= np.abs(min(logweight))
logweight *= sim_parameters["beta"]
logweight /= np.abs(min(logweight))
dt = t[1]-t[0]
tprime = dt * np.cumsum(np.exp(logweight))
min_lag,max_lag = 1,1
n = 1 
lags = np.linspace(min_lag,max_lag,n) 

# MODEL
model = TICA_CV(n_features=X.shape[1])
model.to(device)

In [ ]:
# TRAIN
model.fit_new(X, t, lag=min_lag, logweights=logweight,tprime=tprime)

#-- move the model back to cpu for convenience --# 
model.to('cpu')

#-- print some useful results --#
#print("timescales: ",model.tica.timescales(train_parameters["lag_time"]).detach().cpu().numpy()) 
print("eigenvalues: ",model.tica.evals_.detach().cpu().numpy())

In [ ]:
# plot cvs #
data["cv1"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[0]
data["cv2"] = np.transpose(model(torch.Tensor(X)).detach().cpu().numpy())[1]   

fig,ax = plt.subplots(1,2,figsize=(12,6))
data.plot.hexbin(y="psi",x="phi",C="cv1",cmap="fessa",ax=ax[0])
data.plot.hexbin(y="psi",x="phi",C="cv2",cmap="fessa",ax=ax[1])

fes = np.loadtxt("angles/fes.txt",delimiter=" ")
grid0 = np.loadtxt("angles/grid0.txt",delimiter=" ")
grid1 = np.loadtxt("angles/grid1.txt",delimiter=" ")
bounds = np.arange(0, 60, 5.)
c = ax[0].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c = ax[1].contour(grid0, grid1, fes, bounds, linewidths=3,cmap="gray",linestyles="dashed",
    norm = mpl.colors.BoundaryNorm(bounds, ncolors=len(bounds)-1, clip=False), label="FES [Kj/mol]",
)
c.clabel()
ax[0].grid()
ax[0].set_xlabel(r"$\phi$")
ax[0].set_ylabel(r"$\psi$")
ax[0].set_title("Deep TICA 1")
ax[1].grid()
ax[1].set_xlabel(r"$\phi$")
ax[1].set_ylabel(r"$\psi$")
ax[1].set_title("Deep TICA 2")

fig.savefig(folder+"images/"+"TICAcv_tprime_w-max_divmin.png",transparent=False,facecolor="white",dpi=300)

## $\Delta F$

In [ ]:
'''
#-- each step is 1ps --#
data = load_dataframe(folder+"COLVAR").filter(regex="^phi").to_numpy()[:]
logweight= np.transpose( load_dataframe(folder+"COLVAR").filter(regex="^opes.bias$").to_numpy() )[0][:]
logweight= (logweight-np.max(logweight))*sim_parameters["beta"]
s = data[:]
weight = np.exp(logweight[:])
fes,grid,bounds,error = compute_fes(s, weights=weight,
                                    temp=sim_parameters["temp"],
                                    kbt=sim_parameters["kbt"],
                                    blocks=2,
                                    bandwidth=sim_parameters["bandwidth"],scale_by='range',
                                    plot=False)
ind1 = (grid<0) 
ind2 = (grid>0) 
grid1 = grid[ ind1 ]
grid2 = grid[ ind2 ] 
I1 = integrate.trapz(np.exp(-fes[ind1]*sim_parameters["beta"]), grid1)
I2 = integrate.trapz(np.exp(-fes[ind2]*sim_parameters["beta"]), grid2)
    
res = (1/sim_parameters["beta"])*np.log(I1/I2)
print(res)
'''

In [ ]:
'''
#-- each step is 1ps --#
#-- we are interested in the first 10 ns --#
last = 1*1000*10 #last ns
data = load_dataframe(folder+"COLVAR").filter(regex="^phi").to_numpy()[:last]
logweight= np.transpose( load_dataframe(folder+"COLVAR").filter(regex="^opes.bias$").to_numpy() )[0][:last]
logweight= (logweight-np.max(logweight))*sim_parameters["beta"]
#-- with CLEAR set to 1000 I perform the estimation every ns --#
CLEAR=100

deltaf = np.empty(0)
for el in np.arange(CLEAR,len(data)+CLEAR,CLEAR):
    s = data[:el]
    weight = np.exp(logweight[:el])
    fes,grid,bounds,error = compute_fes(s, weights=weight,
                                        temp=sim_parameters["temp"],
                                        kbt=sim_parameters["kbt"],
                                        blocks=2,
                                        bandwidth=sim_parameters["bandwidth"],scale_by='range',
                                        plot=False)
    ind1 = (grid<0) 
    ind2 = (grid>0) 
    grid1 = grid[ ind1 ]
    grid2 = grid[ ind2 ] 
    I1 = integrate.trapz(np.exp(-fes[ind1]*sim_parameters["beta"]), grid1)
    I2 = integrate.trapz(np.exp(-fes[ind2]*sim_parameters["beta"]), grid2)
    
    deltaf = np.append(deltaf,(1/sim_parameters["beta"])*np.log(I1/I2))

fig,ax = plt.subplots(1,1,figsize=(8,6))
ax.plot(np.arange(len(deltaf)),deltaf,label="Estimate")
res = np.full(len(deltaf),res)
err = np.full(len(deltaf),0.2*kb*sim_parameters["temp"])
ax.plot(np.arange(len(deltaf)),res,linestyle='--',linewidth=3,color="g",label="ref")
ax.fill_between(np.arange(len(deltaf)) , res-err, res+err , color="r",zorder=0,alpha=0.3)
ax.grid()
ax.legend()
ax.set_xlabel(r"$t$ [ps$\times$"+str(CLEAR)+"]")
ax.set_ylabel(r"$\Delta F$ [Kj/mol]")
'''